In [1]:
%cd ../../../../

/home/users/dmoreno2016/ASTROMER/astromer_pe_main/astromer


In [5]:
import toml
import yaml
import os

from src.models.astromer_1 import get_ASTROMER

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
pt_folder = 'presentation/experiments/astromer_1_pe/results/pretraining/P02R01_clean/exp_000_a'

with open(os.path.join(pt_folder, 'config.yaml')) as f:
    model_config = yaml.load(f, Loader=yaml.FullLoader)

with open(os.path.join(pt_folder, 'pe_config.yaml')) as f:
    pe_config = yaml.load(f, Loader=yaml.FullLoader)

astromer = get_ASTROMER(num_layers=model_config['num_layers'],
                        num_heads=model_config['num_heads'],
                        head_dim=model_config['head_dim'],
                        mixer_size=model_config['mixer'],
                        dropout=model_config['dropout'],
                        pe_type=model_config['pe_type'],
                        pe_config=pe_config,
                        window_size=model_config['window_size'],
                        encoder_mode=model_config['encoder_mode'],
                        average_layers=model_config['avg_layers'])

2023-11-08 13:32:27.641627: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Using masking...
Using masking...


In [7]:
astromer.summary()

Model: "ASTROMER-1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mask (InputLayer)           [(None, 200, 1)]             0         []                            
                                                                                                  
 input (InputLayer)          [(None, 200, 1)]             0         []                            
                                                                                                  
 times (InputLayer)          [(None, 200, 1)]             0         []                            
                                                                                                  
 encoder (Encoder)           (None, 200, 256)             660992    ['mask[0][0]',                
                                                                     'input[0][0]',      

In [8]:
for layer in astromer.layers:
    print(f'Name layers: {layer.name}')
    print(f'Number of parameters: {layer.count_params()}')

Name layers: mask
Number of parameters: 0
Name layers: input
Number of parameters: 0
Name layers: times
Number of parameters: 0
Name layers: encoder
Number of parameters: 660992
Name layers: regression
Number of parameters: 769


In [20]:
import datetime

datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

'2023-09-18_01-12-37'

In [9]:
pe_list = ['positional_embedding', 'pos_time_modulation']
pe_weights_names = ['alpha_sin']

for layer in astromer.get_layer('encoder').layers:
    print(f'Name layers: {layer.name}')

    if layer.name.find('att_') != -1:
        print(f'Number of parameters: {layer.count_params()}')

Name layers: inp_transform
Name layers: PosEncoding
Name layers: att_layer_0
Number of parameters: 330112
Name layers: att_layer_1
Number of parameters: 330112
Name layers: dropout_4


In [14]:
astromer.summary()

Model: "ASTROMER-1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mask (InputLayer)              [(None, 200, 1)]     0           []                               
                                                                                                  
 input (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 times (InputLayer)             [(None, 200, 1)]     0           []                               
                                                                                                  
 encoder (Encoder)              (None, 200, 256)     726272      ['mask[0][0]',                   
                                                                  'input[0][0]',         

In [8]:
path_pe_config = 'presentation/experiments/astromer_1_pe/pe_config.toml'
with open(path_pe_config, mode="r") as fp:
    pe_config = toml.load(fp)

In [13]:
pe_config

{'general': {'pe_list': ['positional_embedding',
   'pe_mlp',
   'pe_rnn',
   'pos_time_modulation']},
 'pe': {'base': 1000,
  'c': 1,
  'use_mjd': True,
  'pe_trainable': False,
  'initializer': 'pe_astromer',
  'min_period': 0.01,
  'max_period': 1825},
 'pe_mlp': {'m_layers': 128, 'use_mjd': True},
 'pe_rnn': {'rnn_type': 'gru', 'use_mjd': True},
 'pe_tm': {'T_max': 1500,
  'H': 64,
  'use_mjd': True,
  'pe_trainable': True,
  'initializer': 'random_normal'}}

In [14]:
dict_pe = {
    'general': pe_config['general'],
    'pe': pe_config['pe']
    }
dict_pe

{'general': {'pe_list': ['positional_embedding',
   'pe_mlp',
   'pe_rnn',
   'pos_time_modulation']},
 'pe': {'base': 1000,
  'c': 1,
  'use_mjd': True,
  'pe_trainable': False,
  'initializer': 'pe_astromer',
  'min_period': 0.01,
  'max_period': 1825}}